<a href="https://colab.research.google.com/github/Gk58IISERTvm98/RAG-Application/blob/main/Extract_Data_from_Pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unstructured[all-docs]"

In [ ]:
!pip install pydantic lxml


In [ ]:
!pip install pillow

In [ ]:
!sudo apt-get update

In [ ]:
# !sudo apt-get install libmagic-dev
!sudo apt-get install poppler-utils

In [ ]:
!sudo apt-get install libleptonica-dev libtesseract-dev tesseract-ocr-eng tesseract-ocr-script-latn


In [ ]:
!pip install unstructured-pytesseract

In [ ]:
!pip install tesseract-ocr

In [ ]:
!pip install --upgrade nltk


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
print(nltk.data.path)

['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
raw_pdf_elements= partition_pdf(
    filename="/content/ragmodel.pdf",
    strategy= "hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types= ["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data2"
    )

In [ ]:
raw_pdf_elements

In [ ]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]


for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))

In [ ]:
!pip install -U langchain-chroma langchain-experimental

In [ ]:
!pip install langchain_core
!pip install langchain_openai

In [ ]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
import os
from google.colab import userdata
OPENAI_API_TOKEN=userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

# Table or Text Summary

In [ ]:
tables = []
texts = []
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Table" in str(type(element)):
    tables.append(str(element))
  elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
    texts.append(str(element))


text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts = " ".join(texts)
texts_4k_token = text_splitter.split_text(joined_texts)


In [ ]:
def generate_text_summaries(texts, tables, summarize_texts=False):
  # Prompt
  prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
  These summaries will be embedded and used to retrieve the raw text or table elements. \
  Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
  prompt = ChatPromptTemplate.from_template(prompt_text)
  # Text summary chain
  model = ChatOpenAI(temperature=0, model="gpt-4")
  summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()
  # Initialize empty summaries
  text_summaries = []
  table_summaries = []
  # Apply to text if texts are provided and summarization is requested
  if texts and summarize_texts:
    text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
  elif texts:
    text_summaries = texts

  # Apply to tables if tables are provided
  if tables:
    table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

  return text_summaries, table_summaries



In [ ]:
# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

In [ ]:
table_summaries[0]

# Image Summary

In [ ]:
import base64
import os
from langchain_core.messages import HumanMessage


In [ ]:

def encode_image(image_path):
  """Getting the base64 string"""
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:

def image_summarize(img_base64, prompt):
  """Make image summary"""
  chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)
  msg = chat.invoke(
      [
          HumanMessage(
              content=[
                  {"type": "text", "text": prompt},
                   {
                      "type": "image_url",
                      "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                  },
              ]
          )
      ]
  )
  return msg.content

In [ ]:


def generate_img_summaries(path):
  img_base64_list = []
  # Store image summaries
  image_summaries = []
  # Prompt
  prompt = """You are an assistant tasked with summarizing images for retrieval. \
  These summaries will be embedded and used to retrieve the raw image. \
  Give a concise summary of the image that is well optimized for retrieval."""

  # Apply to images
  for img_file in sorted(os.listdir(path)):
    if img_file.endswith(".jpg"):
      img_path = os.path.join(path, img_file)
      base64_image = encode_image(img_path)
      img_base64_list.append(base64_image)
      image_summaries.append(image_summarize(base64_image, prompt))

  return img_base64_list, image_summaries

In [ ]:
fpath="/content/extracted_data2/figure-17-4.jpg"

img_base64_list,image_summaries=generate_img_summaries(fpath)

In [ ]:
image_summaries[0]

# Multi Vector Retriever

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
  # Initialize the storage layer
  store = InMemoryStore()
  id_key = "doc_id"

  # Create the multi-vector retriever
  retriever = MultiVectorRetriever(
      vectorstore=vectorstore,
      docstore=store,
      id_key=id_key,
  )

  # Helper function to add documents to the vectorstore and docstore
  def add_documents(retriever, doc_summaries, doc_contents):
    doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
    summary_docs = [
        Document(page_content=s, metadata={id_key: doc_ids[i]})
        for i, s in enumerate(doc_summaries)
    ]
    retriever.vectorstore.add_documents(summary_docs)
    retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

  # Add texts, tables, and images
  # Check that text_summaries is not empty before adding
  if text_summaries:
    add_documents(retriever, text_summaries, texts)
  # Check that table_summaries is not empty before adding
  if table_summaries:
    add_documents(retriever, table_summaries, tables)
  # Check that image_summaries is not empty before adding
  if image_summaries:
    add_documents(retriever, image_summaries, images)

  return retriever


In [ ]:
vectorstore = Chroma(
    collection_name="mm_rag_blog", embedding_function=OpenAIEmbeddings()
)

In [ ]:
# Create retriever
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

# RAG

**Build retriever**\
We need to bin the retrieved doc(s) into the correct parts of the GPT-4V prompt template.

In [ ]:
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image


In [ ]:
def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))


In [ ]:
def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


In [ ]:

def is_image_data(b64data):
  """
  Check if the base64 data is an image by looking at the start of the data
  """
  image_signatures = {
      b"\xff\xd8\xff": "jpg",
      b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
      b"\x47\x49\x46\x38": "gif",
      b"\x52\x49\x46\x46": "webp",
  }
  try:
      header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
      for sig, format in image_signatures.items():
          if header.startswith(sig):
              return True
      return False
  except Exception:
      return False

In [ ]:
def resize_base64_image(base64_string, size=(128, 128)):
  img_data = base64.b64decode(base64_string)
  img = Image.open(io.BytesIO(img_data))
  # Resize the image
  resized_img = img.resize(size, Image.LANCZOS)
  # Save the resized image to a bytes buffer
  buffered = io.BytesIO()
  resized_img.save(buffered, format=img.format)
  # Encode the resized image to Base64
  return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [ ]:

def split_image_text_types(docs):
  """
  Split base64-encoded images and texts
  """
  b64_images = []
  texts = []
  for doc in docs:
    # Check if the document is of type Document and extract page_content if so
    if isinstance(doc, Document):
        doc = doc.page_content
    if looks_like_base64(doc) and is_image_data(doc):
        doc = resize_base64_image(doc, size=(1300, 600))
        b64_images.append(doc)
    else:
        texts.append(doc)
  return {"images": b64_images, "texts": texts}

In [ ]:
def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are financial analyst tasking with providing investment advice.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n"
            "Use this information to provide investment advice related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]

In [ ]:
def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


In [ ]:
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [ ]:
# Check retrieval
query = "What are the EV / NTM and NTM rev growth for MongoDB, Cloudflare, and Datadog?"
docs = retriever_multi_vector_img.invoke(query, limit=6)

# We get 4 docs
len(docs)

In [ ]:
# We get back relevant images
plt_img_base64(docs[0])

In [ ]:
image_summaries[3]

**Now let's run RAG and test the ability to synthesize an answer to our question.**

In [ ]:
# Run RAG chain
chain_multimodal_rag.invoke(query)